# Internal Results

In [1]:
### THERE IS 5 extra compounds in DNN, WHY???

from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
import pandas as pd

#datadir = "/data/Shilab/sjwon3789/selectivity_models/models_C33_200/"
datadir = "~/repositories/ai-DR/selectivity_models/models_C33_200/"

def stats(y_true, y_pred):
    Rs = spearmanr(y_true, y_pred)[0]
    Rp = pearsonr(y_true, y_pred)[0]
    R2 = r2_score(y_true, y_pred)
    RMSE = sqrt(mean_squared_error(y_true, y_pred))
    return Rp, Rs, R2, RMSE


def predict_method(df, method):
    y_pred = list(df[method]) 
    y_true = list(df['exp_mean'])

    Rp, Rs, R2, RMSE = stats(y_true, y_pred)
    return Rp, Rs, R2, RMSE

In [2]:
import pandas as pd
import numpy as np

# Define a list of model directories
model_directories = ["models_D2", "models_D3", "models__ratio_D2D3"]

# Create an empty list to store the results for each model
results = []

# Loop through each model directory
for model_dir in model_directories:
    # Create empty lists to store Rp, Rs, R2, and RMSE values for each method
    rp_values_consensus = []
    rs_values_consensus = []
    r2_values_consensus = []
    rmse_values_consensus = []

    rp_values_xgb = []
    rs_values_xgb = []
    r2_values_xgb = []
    rmse_values_xgb = []

    rp_values_rf = []
    rs_values_rf = []
    r2_values_rf = []
    rmse_values_rf = []

    rp_values_dnn = []
    rs_values_dnn = []
    r2_values_dnn = []
    rmse_values_dnn = []

    # Create empty lists to store max values for Rp, Rs, and R2, and min value for RMSE
    max_rp_values_consensus = []
    max_rs_values_consensus = []
    max_r2_values_consensus = []
    min_rmse_values_consensus = []

    max_rp_values_xgb = []
    max_rs_values_xgb = []
    max_r2_values_xgb = []
    min_rmse_values_xgb = []

    max_rp_values_rf = []
    max_rs_values_rf = []
    max_r2_values_rf = []
    min_rmse_values_rf = []

    max_rp_values_dnn = []
    max_rs_values_dnn = []
    max_r2_values_dnn = []
    min_rmse_values_dnn = []

    # Loop through different values of i
    for i in range(10):  # Assuming you have 10 values of i (0 to 9)
        df1_d2 = pd.read_csv(datadir + f"models_D2/model_{i}/pred_reg_xgb_0.00/pred_splits_val{i}", sep="\t")
        df1_d3 = pd.read_csv(datadir + f"models_D3/model_{i}/pred_reg_xgb_0.00/pred_splits_val{i}", sep="\t")
        df1 = pd.DataFrame({'compound': df1_d2['compound'], 'exp_mean': df1_d2['exp_mean'], 'diff_mean': df1_d3['mean'] - df1_d2['mean']}).copy()
        df1 = df1[['compound', 'diff_mean']].copy()
        
        
        df2_d2 = pd.read_csv(datadir + f"models_D2/model_{i}/pred_reg_rf_0.00/pred_splits_val{i}", sep="\t")
        df2_d3 = pd.read_csv(datadir + f"models_D3/model_{i}/pred_reg_rf_0.00/pred_splits_val{i}", sep="\t")
        df2 = pd.DataFrame({'compound': df2_d2['compound'], 'exp_mean': df2_d2['exp_mean'], 'diff_mean': df2_d3['mean'] - df2_d2['mean']}).copy()
        df2 = df2[['compound', 'diff_mean']].copy()
        
        df3_d2 = pd.read_csv(datadir + f"models_D2/model_{i}/pred_reg_dnn_0.00/pred_splits", sep="\t")
        df3_d3 = pd.read_csv(datadir + f"models_D3/model_{i}/pred_reg_dnn_0.00/pred_splits", sep="\t")
        df3 = pd.DataFrame({'compound': df3_d2['compound'], 'exp_mean': df3_d2['exp_mean'], 'diff_mean': df3_d3['mean'] - df3_d2['mean']}).copy()
        df3 = df3[['compound', 'diff_mean']].copy()
        

        # Rename the 'split_0' columns to 'xgb', 'rf', and 'dnn'
        df1.rename(columns={'diff_mean': 'xgb'}, inplace=True)
        df2.rename(columns={'diff_mean': 'rf'}, inplace=True)
        df3.rename(columns={'diff_mean': 'dnn'}, inplace=True)

        # Merge the data frames based on the common columns 'compound' and 'exp_mean'
        df_selectivity = pd.read_csv(datadir + f"models__ratio_D2D3/model_{i}/pred_reg_dnn_0.00/pred_splits", sep="\t")
        merged_df = pd.merge(df1, df2, on=['compound'], how='outer')
        merged_df = pd.merge(merged_df, df3, on=['compound'], how='outer')
        merged_df["consensus"] = merged_df['xgb'] * 1/3 + merged_df["rf"] * 1/3 + merged_df["dnn"] * 1/3
        df_selectivity = df_selectivity[["compound", "exp_mean"]].copy()
        merged_df = pd.merge(merged_df, df_selectivity, on=['compound'], how='inner')


        # Calculate statistical measures per each method and store them
        Rp, Rs, R2, RMSE = predict_method(merged_df, "consensus")
        rp_values_consensus.append(Rp)
        rs_values_consensus.append(Rs)
        r2_values_consensus.append(R2)
        rmse_values_consensus.append(RMSE)
        max_rp_values_consensus.append(np.max(rp_values_consensus))
        max_rs_values_consensus.append(np.max(rs_values_consensus))
        max_r2_values_consensus.append(np.max(r2_values_consensus))
        min_rmse_values_consensus.append(np.min(rmse_values_consensus))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "xgb")
        rp_values_xgb.append(Rp)
        rs_values_xgb.append(Rs)
        r2_values_xgb.append(R2)
        rmse_values_xgb.append(RMSE)
        max_rp_values_xgb.append(np.max(rp_values_xgb))
        max_rs_values_xgb.append(np.max(rs_values_xgb))
        max_r2_values_xgb.append(np.max(r2_values_xgb))
        min_rmse_values_xgb.append(np.min(rmse_values_xgb))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "rf")
        rp_values_rf.append(Rp)
        rs_values_rf.append(Rs)
        r2_values_rf.append(R2)
        rmse_values_rf.append(RMSE)
        max_rp_values_rf.append(np.max(rp_values_rf))
        max_rs_values_rf.append(np.max(rs_values_rf))
        max_r2_values_rf.append(np.max(r2_values_rf))
        min_rmse_values_rf.append(np.min(rmse_values_rf))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "dnn")
        rp_values_dnn.append(Rp)
        rs_values_dnn.append(Rs)
        r2_values_dnn.append(R2)
        rmse_values_dnn.append(RMSE)
        max_rp_values_dnn.append(np.max(rp_values_dnn))
        max_rs_values_dnn.append(np.max(rs_values_dnn))
        max_r2_values_dnn.append(np.max(r2_values_dnn))
        min_rmse_values_dnn.append(np.min(rmse_values_dnn))

    # Create a data frame to store the results for the current model
    model_results_df = pd.DataFrame({
        'Rp_mean': [f"{np.mean(rp_values_xgb):.2f} ({np.std(rp_values_xgb):.2f})",
                    f"{np.mean(rp_values_rf):.2f} ({np.std(rp_values_rf):.2f})",
                    f"{np.mean(rp_values_dnn):.2f} ({np.std(rp_values_dnn):.2f})",
                    f"{np.mean(rp_values_consensus):.2f} ({np.std(rp_values_consensus):.2f})"],
        'Rp_max': [f"{np.max(max_rp_values_xgb):.2f}",
                   f"{np.max(max_rp_values_rf):.2f}",
                   f"{np.max(max_rp_values_dnn):.2f}",
                   f"{np.max(max_rp_values_consensus):.2f}"],
        'Rs_mean': [f"{np.mean(rs_values_xgb):.2f} ({np.std(rs_values_xgb):.2f})",
                    f"{np.mean(rs_values_rf):.2f} ({np.std(rs_values_rf):.2f})",
                    f"{np.mean(rs_values_dnn):.2f} ({np.std(rs_values_dnn):.2f})",
                    f"{np.mean(rs_values_consensus):.2f} ({np.std(rs_values_consensus):.2f})"],
        'Rs_max': [f"{np.max(max_rs_values_xgb):.2f}",
                   f"{np.max(max_rs_values_rf):.2f}",
                   f"{np.max(max_rs_values_dnn):.2f}",
                   f"{np.max(max_rs_values_consensus):.2f}"],
        'R2_mean': [f"{np.mean(r2_values_xgb):.2f} ({np.std(r2_values_xgb):.2f})",
                    f"{np.mean(r2_values_rf):.2f} ({np.std(r2_values_rf):.2f})",
                    f"{np.mean(r2_values_dnn):.2f} ({np.std(r2_values_dnn):.2f})",
                    f"{np.mean(r2_values_consensus):.2f} ({np.std(r2_values_consensus):.2f})"],
        'R2_max': [f"{np.max(max_r2_values_xgb):.2f}",
                   f"{np.max(max_r2_values_rf):.2f}",
                   f"{np.max(max_r2_values_dnn):.2f}",
                   f"{np.max(max_r2_values_consensus):.2f}"],
        'RMSE_mean': [f"{np.mean(rmse_values_xgb):.2f} ({np.std(rmse_values_xgb):.2f})",
                      f"{np.mean(rmse_values_rf):.2f} ({np.std(rmse_values_rf):.2f})",
                      f"{np.mean(rmse_values_dnn):.2f} ({np.std(rmse_values_dnn):.2f})",
                      f"{np.mean(rmse_values_consensus):.2f} ({np.std(rmse_values_consensus):.2f})"],
        'RMSE_min': [f"{np.min(min_rmse_values_xgb):.2f}",
                     f"{np.min(min_rmse_values_rf):.2f}",
                     f"{np.min(min_rmse_values_dnn):.2f}",
                     f"{np.min(min_rmse_values_consensus):.2f}"]
    }, index=['xgb', 'rf', 'dnn', 'consensus'])

    # Append the results for the current model to the overall results list
    results.append((model_dir, model_results_df))

# Create a data frame to store the results for all models
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])
final_results_df.to_csv("output_dir/consensus_selectivity_approach2.csv")
final_results_df

Rp_mean Rp_max      Rs_mean Rs_max  \
models_D2          xgb        0.82 (0.04)   0.87  0.82 (0.03)   0.86   
                   rf         0.83 (0.03)   0.87  0.83 (0.03)   0.87   
                   dnn        0.83 (0.03)   0.86  0.83 (0.02)   0.87   
                   consensus  0.85 (0.03)   0.89  0.85 (0.03)   0.88   
models_D3          xgb        0.82 (0.04)   0.87  0.82 (0.03)   0.86   
                   rf         0.83 (0.03)   0.87  0.83 (0.03)   0.87   
                   dnn        0.83 (0.03)   0.86  0.83 (0.02)   0.87   
                   consensus  0.85 (0.03)   0.89  0.85 (0.03)   0.88   
models__ratio_D2D3 xgb        0.82 (0.04)   0.87  0.82 (0.03)   0.86   
                   rf         0.83 (0.03)   0.87  0.83 (0.03)   0.87   
                   dnn        0.83 (0.03)   0.86  0.83 (0.02)   0.87   
                   consensus  0.85 (0.03)   0.89  0.85 (0.03)   0.88   

                                  R2_mean R2_max    RMSE_mean RMSE_min  
models_D2          xgb        0.67 (0.07)   0.75  0.58 (0.06)     0.50  
                   rf         0.69 (0.05)   0.76  0.56 (0.05)     0.50  
                   dnn        0.67 (0.06)   0.74  0.58 (0.05)     0.52  
                   consensus  0.72 (0.06)   0.78  0.53 (0.06)     0.47  
models_D3          xgb        0.67 (0.07)   0.75  0.58 (0.06)     0.50  
                   rf         0.69 (0.05)   0.76  0.56 (0.05)     0.50  
                   dnn        0.67 (0.06)   0.74  0.58 (0.05)     0.52  
                   consensus  0.72 (0.06)   0.78  0.53 (0.06)     0.47  
models__ratio_D2D3 xgb        0.67 (0.07)   0.75  0.58 (0.06)     0.50  
                   rf         0.69 (0.05)   0.76  0.56 (0.05)     0.50  
                   dnn        0.67 (0.06)   0.74  0.58 (0.05)     0.52  
                   consensus  0.72 (0.06)   0.78  0.53 (0.06)     0.47

In [3]:
merged_df

,compound,xgb,rf,dnn,consensus,exp_mean
0,CHEMBL271989,-0.826973,-0.633921,-1.392054,-0.950983,-0.43
1,CHEMBL245816,-0.517804,-0.517476,-0.622990,-0.552757,0.00
2,CHEMBL563131,1.649311,1.710425,1.738039,1.699258,1.60
3,CHEMBL399164,1.333731,1.452978,1.015602,1.267437,1.32
4,CHEMBL61717,2.307969,2.225493,1.582863,2.038775,2.20
...,...,...,...,...,...,...
192,CHEMBL4293307,0.340413,0.293514,0.339755,0.324561,-0.02
193,CHEMBL4071868,0.550781,0.396136,-0.201907,0.248337,0.31
194,CHEMBL2387636,0.645008,1.050611,1.334081,1.009900,0.75
195,CHEMBL242218,2.161079,2.428332,2.548545,2.379319,2.14
